In [ ]:
# Imports
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os
import scipy as sp
from scipy.constants import hbar, physical_constants
from scipy.stats import chi2
import seaborn as sns
from tqdm import tqdm

# Retrieve physical constants
e_au = physical_constants["atomic unit of charge"][0]

# Conversion factors
s_to_eVminus1 = e_au / hbar

In [ ]:
# Define the data folder
data_folder = "sgrdata"
file_black = os.path.join(data_folder, "Dataset_black.csv")
file_blue1 = os.path.join(data_folder, "Dataset_blue1.csv")
file_blue2 = os.path.join(data_folder, "Dataset_blue2.csv")
file_red1 = os.path.join(data_folder, "Dataset_red1.csv")
file_red2 = os.path.join(data_folder, "Dataset_red2.csv")

In [ ]:
# Read the data
data_black = pd.read_csv(file_black, names=["time", "phi"], header=None)
data_blue1 = pd.read_csv(file_blue1, names=["time", "phi"], header=None)
data_blue2 = pd.read_csv(file_blue2, names=["time", "phi"], header=None)
data_red1 = pd.read_csv(file_red1, names=["time", "phi"], header=None)
data_red2 = pd.read_csv(file_red2, names=["time", "phi"], header=None)

# Calculate average values for blue and red datasets
data_blue = pd.DataFrame({
    "time": (data_blue1["time"] + data_blue2["time"]) / 2,
    "phi": (data_blue1["phi"] + data_blue2["phi"]) / 2,
    "sigma": abs(data_blue1["phi"] - data_blue2["phi"]) / 2
})

data_red = pd.DataFrame({
    "time": (data_red1["time"] + data_red2["time"]) / 2,
    "phi": (data_red1["phi"] + data_red2["phi"]) / 2,
    "sigma": abs(data_red1["phi"] - data_red2["phi"]) / 2
})

# Add sigma column to black data
data_black["sigma"] = 3

# Concatenate the data
data_total = pd.concat([data_black, data_blue, data_red])
data_total = data_total.sort_values(by="time").reset_index(drop=True)

In [ ]:
# Read the pickle file
interp_file = os.path.join(data_folder, "interp_datanew.pkl")
with open(interp_file, "rb") as f:
    interp_data = pickle.load(f)

# Extract the parameters
params_data = np.loadtxt("paralist_all_SgrltNE.dat")
params = []
for i in range(len(params_data) // 20):
    m = params_data[i * 20, 1]
    epsilon = params_data[i * 20, 2]
    params.append([m, epsilon])

# Convert to numpy array
params = np.array(params)

In [ ]:
def mass_to_period(m):
    # Convert mass to period
    period_hr = 2 * np.pi / m / s_to_eVminus1 / 3600
    return period_hr

def period_to_mass(period_hr):
    # Convert period to mass
    m = 2 * np.pi / period_hr / s_to_eVminus1 / 3600
    return m

def extract_data_obs(data):
    # Extract data from observation
    times_obs = data["time"].to_numpy()
    phis_obs = data["phi"].to_numpy()
    sigmas_obs = data["sigma"].to_numpy()
    return times_obs, phis_obs, sigmas_obs

def compute_chi_sq(phis_obs, sigmas_obs, phis_theory):
    # Compute chi-squared value
    chi_sq = np.sum((phis_obs - phis_theory) ** 2 / sigmas_obs ** 2)
    return chi_sq

def compute_chi_sqs_params(i, params, times_obs, phis_obs, sigmas_obs, interp_data, phi_bkgs):
    # Compute chi-squared values for a single parameter set
    chi_sqs = []

    # Extract the parameters
    m, epsilon = params[i]

    # Calculate the period in hours
    period_hr = mass_to_period(m)

    # Generate initial phases
    phases = np.linspace(0, period_hr, 100)

    # Get the theoretical best-fit function
    delta_phi_func = interp_data[i]

    for phase in phases:
        # Time shift calculation
        times = (times_obs + phase) % period_hr

        # Get the theoretical delta phi values
        delta_phis = delta_phi_func(times)

        # Calculate the theoretical phi values
        for phi_bkg in phi_bkgs:
            phis_theory = delta_phis + phi_bkg

            # Calculate chi-squared values
            chi_sq = compute_chi_sq(phis_obs, sigmas_obs, phis_theory)
            chi_sqs.append([chi_sq, phase, phi_bkg])

    return chi_sqs

def compute_chi_sqs_all(params, times_obs, phis_obs, sigmas_obs, interp_data, phi_bkgs, n_jobs=-1):
    # Compute chi-squared values for all parameter sets
    chi_sqs_all = Parallel(n_jobs=n_jobs)(
        delayed(compute_chi_sqs_params)(i, params, times_obs, phis_obs, sigmas_obs, interp_data, phi_bkgs) 
        for i in tqdm(range(len(params)))
    )

    return chi_sqs_all

def extract_chi_sqs_min(chi_sqs_all):
    # Initialize an empty array to store the results
    chi_sqs_min = np.empty((len(chi_sqs_all), 3))
    
    for i in range(len(chi_sqs_all)):
        # Find the index of the minimum chi-squared value for the i-th parameter combination
        min_idx = np.argmin(chi_sqs_all[i][:, 0])

        # Store the minimum chi-squared value and the corresponding phase and phi_bkg
        chi_sqs_min[i] = chi_sqs_all[i][min_idx]
    
    return chi_sqs_min

In [ ]:
# Define the datasets with labels
datasets = {
    "black": {"data": data_black, "label": "CARMA"},
    "blue": {"data": data_blue, "label": "SMTL-CARMAR"},
    "red": {"data": data_red, "label": "SMTR-CARMAL"},
    "total": {"data": data_total, "label": "All"}
}

# Define the background phases
phi_bkgs = np.linspace(0, 180, 100)

for colour, info in datasets.items():
    # Define the filename
    filename = f"chi_sqs_all_{colour}.npy"

    # Check if file already exists to avoid recomputing
    if os.path.exists(filename):
        print(f"File {filename} already exists. Skipping computation.")
        continue
    
    data = info["data"]
    label = info["label"]
    
    # Extract observational data
    times_obs, phis_obs, sigmas_obs = extract_data_obs(data)
    
    # Compute chi-squared values for all parameter combinations
    chi_sqs_all = compute_chi_sqs_all(params, times_obs, phis_obs, sigmas_obs, interp_data, phi_bkgs)
    
    # Save the results to a .npy file
    np.save(filename, chi_sqs_all)
    print(f"Saved {filename} for {label}.")

In [ ]:
# Load the chi-squared values
chi_sqs_all_black = np.load("chi_sqs_all_black.npy", allow_pickle=True)
chi_sqs_all_blue = np.load("chi_sqs_all_blue.npy", allow_pickle=True)
chi_sqs_all_red = np.load("chi_sqs_all_red.npy", allow_pickle=True)
chi_sqs_all_total = np.load("chi_sqs_all_total.npy", allow_pickle=True)

# Extract minimum chi-squared values for each data set
chi_sqs_min_black = extract_chi_sqs_min(chi_sqs_all_black)
chi_sqs_min_blue = extract_chi_sqs_min(chi_sqs_all_blue)
chi_sqs_min_red = extract_chi_sqs_min(chi_sqs_all_red)
chi_sqs_min_total = extract_chi_sqs_min(chi_sqs_all_total)

In [ ]:
# Define threshold for 95% CL of one-sided chi-squared distribution of 1 DoF
threshold = chi2.ppf(0.95, df=1)

# Extract chi-squared values and parameters
chi_sqs_min = chi_sqs_min_total[:, 0]
masses = params[:, 0]
epsilons = params[:, 1]

# Verify structure of input data
unique_masses = np.unique(masses)
unique_epsilons = np.unique(epsilons)
num_masses = len(unique_masses)
num_epsilons = len(unique_epsilons)
assert num_masses * num_epsilons == len(params), "Parameter array size mismatch"

# Reshape data to match unique masses and epsilons
chi_sqs_per_mass = chi_sqs_min.reshape(num_masses, num_epsilons)
epsilons_per_mass = epsilons.reshape(num_masses, num_epsilons)

# Compute upper limits
chi_sq_min_per_mass = np.zeros(num_masses)
epsilon_min_per_mass = np.zeros(num_masses)
upper_limit_epsilons = np.zeros(num_masses)

for i in range(num_masses):
    chi_sqs = chi_sqs_per_mass[i]
    epsilons = epsilons_per_mass[i]
    
    # Find minimum chi-squared and corresponding epsilon
    chi_sq_min_idx = np.argmin(chi_sqs)
    chi_sq_min = chi_sqs[chi_sq_min_idx]
    chi_sq_min_per_mass[i] = chi_sq_min
    epsilon_min = epsilons[chi_sq_min_idx]
    epsilon_min_per_mass[i] = epsilon_min
    chi_sq_threshold = chi_sq_min + threshold
    
    # Sort by epsilon in ascending order
    sort_idx = np.argsort(epsilons)
    epsilons_sorted = epsilons[sort_idx]
    chi_sqs_sorted = chi_sqs[sort_idx]
    
    # Find the index of epsilon_min in sorted epsilon array
    sort_idx_min = np.where(epsilons_sorted == epsilon_min)[0][0]
    
    # Increase epsilon from epsilon_min until chi-squared exceeds threshold
    upper_limit_epsilons[i] = np.nan
    for j in range(sort_idx_min, len(epsilons_sorted)):
        if chi_sqs_sorted[j] > chi_sq_threshold:
            upper_limit_epsilons[i] = epsilons_sorted[j]
            break

# Combine and print results
results = np.column_stack((unique_masses, chi_sq_min_per_mass, epsilon_min_per_mass, upper_limit_epsilons))
print("Mass | Min chi-sq | Min epsilon | 95% CL upper limit on epsilon")
for mass, min_chi_sq, epsilon_min, epsilon_limit in results:
    print(f"{mass:.2e} | {min_chi_sq:.4f} | {epsilon_min:.2e} | {epsilon_limit:.2e}")

In [ ]:
# Create a figure with multiple subplots
fig, axes = plt.subplots(4, 4, figsize=(20, 20))
axes = axes.flatten()

# Plot chi-squared values againt coupling for each mass
for i, mass in enumerate(unique_masses):
    if i >= len(axes):
        break
    
    # Extract precomputed values for the given mass
    chi_sqs = chi_sqs_per_mass[i] # Chi-squared values for this mass
    epsilons = epsilons_per_mass[i] # Epsilon values for this mass
    chi_sq_min = chi_sq_min_per_mass[i] # Precomputed minimum chi-squared
    epsilon_min = epsilon_min_per_mass[i] # Precomputed epsilon at minimum chi-squared
    upper_limit = upper_limit_epsilons[i] # Precomputed 95% CL upper limit on epsilon
    
    # Compute the threshold for this mass
    chi_sq_threshold = chi_sq_min + threshold
    
    # Create plot on the corresponding subplot
    axes[i].semilogx(epsilons, chi_sqs, "o-", color="blue", markersize=3)
    
    # Add horizontal line at 95% CL threshold and minimum chi-squared
    axes[i].axhline(y=chi_sq_threshold, color="r", linestyle="--", label=f"95% CL ($\\chi^2_\\text{{min}}$ + {threshold:.2f})")
    axes[i].axhline(y=chi_sq_min, color="g", linestyle="--", label=r"$\chi^2_\text{min}$")
    
    # Highlight min and upper limit points
    axes[i].plot(epsilon_min, chi_sq_min, "go", markersize=3)
    
    # Add labels and title
    axes[i].set_xlabel(r"Coupling $\epsilon$", fontsize=12)
    axes[i].set_ylabel(r"$\chi^2$", fontsize=12)
    mass_coefficient = mass / 10**np.floor(np.log10(mass))
    mass_exponent = int(np.floor(np.log10(mass)))
    axes[i].set_title(rf"$m={mass_coefficient:.2f} \times 10^{{{mass_exponent}}}$ eV", fontsize=14)
    axes[i].legend(loc="upper right", fontsize=10)
    
    # Plot the upper limit if it exists
    if not np.isnan(upper_limit):
        axes[i].axvline(x=upper_limit, color="r", linestyle="--")

# Hide unused subplots
for j in range(len(unique_masses), len(axes)):
    axes[j].axis("off")

# Adjust the spacing
plt.tight_layout()

# Show the plot
plt.show()